# Encoding classical data into qubits using Pennylane and Qiskit

In [1]:
# Standard Pennylane imports
import pennylane as qml
from pennylane import numpy as np

# Basis Encoding using PennyLane

In [2]:
# Import Basis encoding template
# Pennylane provides in-built encoding templates for different types of encodings like Basis, Amplitude and Angle
from pennylane.templates.embeddings import BasisEmbedding

In [3]:
#Sample data point containing 3 features
data = np.array([1,0,1])
# In Basis encoding, the number of qubits used is equal to the total number of bits in the features we have in our dataset
# For example, lets say we have 2 datapoints with feature values (01,01) and (11,10).
# Then, basis encoding shall result in a superposition of |0101> and |1110> with equal probabilities.
# Basis encoding can ONLY encode Binary data since a qubit has just two basis states - 0 and 1.

# Create the default simulator device in Pennylane with 3 qubits.
dev = qml.device('default.qubit', wires=3)

# qnode decorator for the circuit to let Pennylane know that this is a quantum device
@qml.qnode(dev)
def circuit(data):
    BasisEmbedding(features=data,wires=range(3))
    # Check if the input data has been correctly encoded or not by measuring across the Z axis.
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2))

In [4]:
circuit(data)

(tensor(-1., requires_grad=True),
 tensor(1., requires_grad=True),
 tensor(-1., requires_grad=True))

# Angle encoding with Pennylane

In [5]:
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize
np.random.seed(42)

In [6]:
# Angle encoding is more expressive,and therefore can handle floating point data. The principle here is that
# the qubits are rotated about a particular axis with the angle equal to the floating point value of the input data
# Here the number of qubits required is equal to the number of features in the dataset.
x, Y = load_iris().data, load_iris().target
x, Y = shuffle(x,Y)

x = x[:5]
Y = Y[:5]

In [7]:
print(x,Y)

[[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]] [1 0 2 1 1]


In [8]:
data = normalize(x)
print(data)

[[0.73659895 0.33811099 0.56754345 0.14490471]
 [0.8068282  0.53788547 0.24063297 0.04246464]
 [0.70600618 0.2383917  0.63265489 0.21088496]
 [0.73350949 0.35452959 0.55013212 0.18337737]
 [0.76467269 0.31486523 0.53976896 0.15743261]]


In [9]:
# Import Angle encoding template
from pennylane.templates.embeddings import AngleEmbedding

In [10]:
num_qubits = len(data[0]) # No. of qubits= No. of features
dev = qml.device('default.qubit', wires=num_qubits)

@qml.qnode(dev)
def circuit(data):
    # First we make all gates into superpositions by applying a Hadamard gate to them.
    # The Hadamard gate is a single qubit gate that creates an equal superposition of two basis states.
    # Thus all the qubits will be in a superposition of the |0> and |1> basis states after this operation.
    # Visually, we can say that all qubits in the Bloch sphere are on the equator and pointing in the positive X direction at this stage
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    # After this, we pass on the qubits to the AngleEmbedding template, which by default rotates the qubits about the 'X' axis
    # with the value provided in the data. This as you can imagine, will not change anything and the measurement about the 'Z' axis
    # will just be equal to 0. Hence, we change this rotation axis to 'Y', so that we can observe the rotation and the measurement 
    # about the Z axis will have a meaningful value.
    AngleEmbedding(features=data, wires=range(num_qubits), rotation='Y')    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

In [11]:
circuit(data[0])

(tensor(-0.67177245, requires_grad=True),
 tensor(-0.33170563, requires_grad=True),
 tensor(-0.53756225, requires_grad=True),
 tensor(-0.14439814, requires_grad=True))

In [12]:
# Encoding all data
# Instead of our circuit just accepting one datapoint,we can encode our entire input feature space into this circuit by
# repeatedly applying rotations one after the other on the qubits. This is done by introducing a for loop in the AngleEmbedding template.
@qml.qnode(dev)
def circuit(data):
    for i in range(num_qubits):
        qml.Hadamard(wires=i)
    for i in range(len(data)):
        AngleEmbedding(features=data[i], wires=range(num_qubits), rotation='Y')    
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1)), qml.expval(qml.PauliZ(2)), qml.expval(qml.PauliZ(3))

In [13]:
circuit(data)

(tensor(0.56960308, requires_grad=True),
 tensor(-0.97740396, requires_grad=True),
 tensor(-0.57357236, requires_grad=True),
 tensor(-0.67359663, requires_grad=True))

# Angle encoding with Qiskit

In [14]:
# Import the entire Qiskit library.
from qiskit import *
# The ZFeaturemap is an inbuilt template circuit which already has the Hadamard gates and the rotations pre-populated.
from qiskit.circuit.library import ZFeatureMap

In [15]:
# We initialize this template by specifying the number of qubits and the number of repetitions we want for this block
featuremap_circ = ZFeatureMap(4, reps=1)

In [16]:
# The .decompose() function will show all the component gates in this ZFeatureMap circuit block.
# We can see that this has bee initialized with a template data of x[0], x[1], x[2], x[3]
print(featuremap_circ.decompose())

     ┌───┐┌─────────────┐
q_0: ┤ H ├┤ P(2.0*x[0]) ├
     ├───┤├─────────────┤
q_1: ┤ H ├┤ P(2.0*x[1]) ├
     ├───┤├─────────────┤
q_2: ┤ H ├┤ P(2.0*x[2]) ├
     ├───┤├─────────────┤
q_3: ┤ H ├┤ P(2.0*x[3]) ├
     └───┘└─────────────┘


In [17]:
# We assign the values of x[0], x[1], x[2], x[3] using the assign_parameters() function. The /2 is introduced
# to counter the multiplication by 2 in the default template.
circ_data0 = featuremap_circ.assign_parameters(data[0]/2)

print(circ_data0.decompose())

     ┌───┐┌───────────┐ 
q_0: ┤ H ├┤ P(0.7366) ├─
     ├───┤├───────────┴┐
q_1: ┤ H ├┤ P(0.33811) ├
     ├───┤├────────────┤
q_2: ┤ H ├┤ P(0.56754) ├
     ├───┤├───────────┬┘
q_3: ┤ H ├┤ P(0.1449) ├─
     └───┘└───────────┘ 


In [18]:
# We can expand this circuit by attaching the next datapoint to the existing circuit using the compose() function.
circ_data1 = circ_data0.compose(featuremap_circ.assign_parameters(data[1]/2))

print(circ_data1.decompose())

     ┌───┐┌───────────┐ ┌───┐ ┌────────────┐
q_0: ┤ H ├┤ P(0.7366) ├─┤ H ├─┤ P(0.80683) ├
     ├───┤├───────────┴┐├───┤ ├────────────┤
q_1: ┤ H ├┤ P(0.33811) ├┤ H ├─┤ P(0.53789) ├
     ├───┤├────────────┤├───┤ ├────────────┤
q_2: ┤ H ├┤ P(0.56754) ├┤ H ├─┤ P(0.24063) ├
     ├───┤├───────────┬┘├───┤┌┴────────────┤
q_3: ┤ H ├┤ P(0.1449) ├─┤ H ├┤ P(0.042465) ├
     └───┘└───────────┘ └───┘└─────────────┘


In [19]:
# This job can be simplified with the help of a for loop.
circ = QuantumCircuit(4)

for i in range(len(data)):
    circ = circ.compose(featuremap_circ.assign_parameters(data[i]/2))

print(circ.decompose())

     ┌───┐┌───────────┐ ┌───┐ ┌────────────┐┌───┐┌────────────┐┌───┐»
q_0: ┤ H ├┤ P(0.7366) ├─┤ H ├─┤ P(0.80683) ├┤ H ├┤ P(0.70601) ├┤ H ├»
     ├───┤├───────────┴┐├───┤ ├────────────┤├───┤├────────────┤├───┤»
q_1: ┤ H ├┤ P(0.33811) ├┤ H ├─┤ P(0.53789) ├┤ H ├┤ P(0.23839) ├┤ H ├»
     ├───┤├────────────┤├───┤ ├────────────┤├───┤├────────────┤├───┤»
q_2: ┤ H ├┤ P(0.56754) ├┤ H ├─┤ P(0.24063) ├┤ H ├┤ P(0.63265) ├┤ H ├»
     ├───┤├───────────┬┘├───┤┌┴────────────┤├───┤├────────────┤├───┤»
q_3: ┤ H ├┤ P(0.1449) ├─┤ H ├┤ P(0.042465) ├┤ H ├┤ P(0.21088) ├┤ H ├»
     └───┘└───────────┘ └───┘└─────────────┘└───┘└────────────┘└───┘»
«     ┌────────────┐┌───┐┌────────────┐
«q_0: ┤ P(0.73351) ├┤ H ├┤ P(0.76467) ├
«     ├────────────┤├───┤├────────────┤
«q_1: ┤ P(0.35453) ├┤ H ├┤ P(0.31487) ├
«     ├────────────┤├───┤├────────────┤
«q_2: ┤ P(0.55013) ├┤ H ├┤ P(0.53977) ├
«     ├────────────┤├───┤├────────────┤
«q_3: ┤ P(0.18338) ├┤ H ├┤ P(0.15743) ├
«     └────────────┘└───┘└────────────┘


# Higher order feature encoding using Qiskit

In [20]:
# The ZZFeatureMap is another template that can be used to have higher order encoding where we rotate the qubits with
# the multiplication of the angles. This concept is borrowed from classical machine learning where if we transform the
# input parameters into a higher dimension, then the data might be more linearly separable. This same concept is what the
# ZZFeaturemap does. It enhances the existing data into an expanded space.
from qiskit.circuit.library import ZZFeatureMap

In [21]:
featuremap_circ = ZZFeatureMap(2, reps=1)

In [22]:
print(featuremap_circ.decompose())

     ┌───┐┌─────────────┐                                          
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■──
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├
     └───┘└─────────────┘└───┘└──────────────────────────────┘└───┘
